In [ ]:
from pathlib import Path

import networkx as nx
import pandas as pd

from networkx.algorithms.traversal import bfs_tree

"""
Load data from file
"""

# set up some paths for ins and outs
base_directory = Path(__file__).parent.parent.absolute()
source_data_path = base_directory / "data" / "raw" / "etymwn.tsv"
processed_directory = base_directory / "data" / "processed"
output_graph_yaml_file = processed_directory / "graph_output.yaml"
output_graph_dot_file = processed_directory / "graph_output.dot"
output_graph_png = processed_directory / "graph_output.png"

# load from CSV
cleaned_df = pd.read_csv(
    source_data_path, sep="\t", names=["source_node", "edge_type", "target_node"]
)



# Cleaning notes

## Relationship types found in source data

Relationships are recorded bidirectionally. So a root word will link to its derivatives, and each derivative will link back to the root. To simplify the graph,
I will drop edges that point from derivatives to roots, since that information is already encoded in the root-to-leaf edge and networkx can handle bidirectional
traversal without requiring multiple edges to link the same pair of nodes.

Below are the types of relationships extracted from the data, with comments indicating their directionality.

```python

"""
<- A is the source of B
-> B is the source of A
"""

rel_types = [
    "rel:etymology", # ->
    "rel:etymological_origin_of", # <-
    "rel:is_derived_from", # ->
    "rel:has_derived_form", # <-
    "rel:etymologically_related", # <->
    "rel:variant:orthography", # <->
]

# source data also includes a handful of malformed values, which should be dropped or replaced 
cleaned_df = cleaned_df.replace("rel:etymologically", "rel:etymologically_related").replace("rel:derived", "rel:is_derived_from")

```


In [ ]:

"""
Clean data
"""

# filter out bidirectional relationships and select one directionality to normalize the graph
# I would normally clean to fix a handful of malformed tags as below but we are dropping those edge types anyway
# so instead we will stick to the edge types that point from root words to derived words
root_first_rel_types = ["rel:etymological_origin_of", "rel:has_derived_form"]
cleaned_df = cleaned_df.loc[(cleaned_df["edge_type"].isin(root_first_rel_types))]
cleaned_df[["source_language", "source_word"]] = cleaned_df.source_node.str.split(
    ": ", expand=True
)

# there are a handful of nodes that include strange characters or a :Category: tag that introduces a third
# column for no reason. This data is uninteresting so we can just ignore it and no include it in the graph when we construct it
cleaned_df[["target_language", "target_word", "crud"]] =  cleaned_df.target_node.str.split(": ", expand=True)


In [ ]:

"""
Get list of unique languages in data set. Not super necessary but helpful for understanding the likely subgraph structure. I would guess that the individual
languages will be highly connected/clustered. I suspect the boundaries will blur a bit around the proto- and ancient languages, particularly for languages with
many ancestors in the data set, e.g., Latin
"""

unique_languages = set(cleaned_df["source_language"].unique()).union(set(cleaned_df["target_language"].unique()))
print(unique_languages)

In [ ]:

"""
Construct networkx graph
"""

# start with directed so we can preserve directionality data, retaining the option to convert to undirected later to use networkx undirected algorithms
graph = nx.from_pandas_edgelist(
    cleaned_df,
    edge_attr=[
        "edge_type",
        "source_language",
        "source_word",
        "target_language",
        "target_word",
    ],
    source="source_node",
    target="target_node",
    create_using=nx.DiGraph,
)
print(nx.info(graph))



# What next?

Now that we've loaded the data into a graph, it's time to do some actual analysis. But to do so, we need to define the problem more clearly. Otherwise, there is
an intractable amount of data for many graph algorithms. Pruning to relevant subgraphs would be desirable as an initial post-processing step.

I am primarily interested in English language entries. However, ~~many~~ all of the English words are derived from non-English words. It would be good to prune
entries that are not etymological roots of English words.

My initial instinct is to trim any descendant nodes of English words that are in other languages. Then, we can run BFS from each English node to gather its
ancestors, knowing the descendant nodes have already been trimmed. This could accidentally exclude relevant data if there are derivation paths that jump from
English to another language and then back, but that's an interesting question in and of itself and might be worth investigating as preliminary matter before
pursuing this approach. 

However, I suspect the BFS approach may be extremely inefficient and that it will be necessary to reduce the size of the search space to something more
tractable. 

So. What exactly are we looking for?

- Nodes with the "eng:" prefix
- Nodes that are direct and indirect ancestors of the English nodes



In [ ]:
"""
Undirected graph analysis
"""

# convert to undirected so we can apply undirected algorithms
undirected_graph = graph.to_undirected()
print(nx.info(undirected_graph))

# check for connected components. may be a way to prune subgraphs that do not relate to English etymology
connected_components = undirected_graph.connected_components()
print(len(connected_components))


In [ ]:

"""
Create English-related subgraph
"""

# grab the nodes that have the eng tag, then build the connected graph from those
english_nodes = [n for n in graph.nodes() if n[0:3] == 'eng']
nodes_to_add = []
# add nodes inside loop to avoid having to flatten later
[nodes_to_add.extend(bfs_tree(graph, source=node)) for node in english_nodes]

english_graph = graph.subgraph(nodes_to_add)
print(nx.info(english_graph))


In [ ]:

nodes_by_degree = sorted(english_graph.degree, key=lambda x: x[1], reverse=True)
print(nodes_by_degree[0:99])
